In [135]:
# Re-run this cell whenever parser.py is changed
import importlib
importlib.reload(parser)

<module 'parser' from '/Users/ivanprigarin/Library/Mobile Documents/com~apple~CloudDocs/cloud-files/1_study/cluster/code/mln-project/tool/parser.py'>

In [15]:
import parser
from pprint import pprint

In [16]:
mln_data = parser.parse_mln('network.txt')
pprint(mln_data)

Parsing network configuration file network.txt
{'actors': (['1', '2', '3'], 3),
 'edges': ({'dir': [False, False, False, False, False],
            'from_actor': ['1', '1', '1', '1', '2'],
            'from_layer': ['1', '1', '2', '2', '2'],
            'to_actor': ['2', '3', '2', '3', '3'],
            'to_layer': ['1', '1', '2', '2', '2']},
           [[('1', '2'), ('2', '1'), ('1', '3'), ('3', '1')],
            [('1', '2'),
             ('2', '1'),
             ('1', '3'),
             ('3', '1'),
             ('2', '3'),
             ('3', '2')]]),
 'layers': (['1', '2'], 2)}


In [69]:
language = parser.parse_language_file('toy-example.mln')
pprint(lang)

Parsing MLN language file toy-example.mln
{'initial conditions': ['1 = AI', '2 = US', '3 = US'],
 'parameters': ['delta = 1', 'mu = 1', 'betaA = 1', 'betaU = 3', 'lamb = 1'],
 'rules': ['AS -> US @ delta',
           'AI -> AS @ mu',
           'AS =1 AI -> AI =1 AI @ betaA',
           'US =1 AI -> AI =1 AI @ betaU',
           'US =2 AI -> AS =2 AI @ lamb',
           'US =2 AS -> AS =2 AS @ lamb'],
 'simOptions': ['n = 1000', 't = 30'],
 'states': ['AS', 'AI', 'US'],
 'views': ['AI', 'AS', 'US']}


In [130]:
def parse_signatures(mln_data, language):
    signatures = ['/* Signatures */']
    
    states = []
    for state in language['states']:
        states.append(state)
    states = '{' + ', '.join(states) + '}'

    for i, actor in enumerate(mln_data['actors'][0]):
        sites = []
        for j, layer in enumerate(mln_data['edges'][1]):
            for edge in layer:
                if edge[0] == str(i+1):
                    sites.append(f'l{j+1}v{edge[1]}')
        sites = ', '.join(sites)
                                 
        kappa_signature = f'%agent: V{i+1}(state{states}, {sites})'
        signatures.append(kappa_signature)
        
        
    signatures = '\n'.join(signatures)
    
    return signatures

In [131]:
print(parse_signatures(mln_data, language))

/* Signatures */
%agent: V1(state{AS, AI, US}, l1v2, l1v3, l2v2, l2v3)
%agent: V2(state{AS, AI, US}, l1v1, l2v1, l2v3)
%agent: V3(state{AS, AI, US}, l1v1, l2v1, l2v2)


In [31]:
def parse_variables(language):
    variables = ['/* Variables */']
    
    for param in language['parameters']:
        var_name, var_value = param.split('=')
        var = f"%var: '{var_name.strip()}' {var_value.strip()}"
        variables.append(var)
    variables = '\n'.join(variables)
    
    return variables

In [32]:
print(parse_variables(language))

/* Variables */
%var: 'delta' 1
%var: 'mu' 1
%var: 'betaA' 1
%var: 'betaU' 3
%var: 'lamb' 1


In [36]:
def parse_observables(mln_data, language):
    observables = ['/* Observables */']
    
    for view in language['views']:
        components = []
        for i, actor in enumerate(mln_data['actors'][0]):
            components.append(f'|V{i+1}(state{{{view}}})|')
        obs = f"%obs: '{view}' " + ' + '.join(components)
        observables.append(obs)
    
    observables = '\n'.join(observables)
    
    return observables

In [37]:
print(parse_observables(mln_data, language))

/* Observables */
%obs: 'AI' |V1(state{AI})| + |V2(state{AI})| + |V3(state{AI})|
%obs: 'AS' |V1(state{AS})| + |V2(state{AS})| + |V3(state{AS})|
%obs: 'US' |V1(state{US})| + |V2(state{US})| + |V3(state{US})|


In [64]:
def parse_initial_conditions(mln_data, language):
    n = ''
    for line in language['simOptions']:
        option, value = line.split('=')
        if option.strip() == 'n':
            n = value.strip()
            break
            
    i_c = ['/* Initial conditions */',
           f'%init: {n} (']
    
    initial_states = [x.split('=') for x in language['initial conditions']]

    for i, actor in enumerate(mln_data['actors'][0]):
        sites = []
        site_labels = []
        for j, layer in enumerate(mln_data['edges'][1]):
            for edge in layer:
                if edge[0] == str(i+1):
                    # weird way to keep site labels consistent.
                    # will need a rewrite
                    if i+1 <= int(edge[1]):
                        site_label = f'{j+1}{i+1}{edge[1]}'
                    else:
                        site_label = f'{j+1}{edge[1]}{i+1}'
                        
                    sites.append(f'l{j+1}v{edge[1]}[{site_label}]')
        sites = ', '.join(sites)
        
        initial_state = ''
        for state in initial_states:
            if state[0].strip() == str(i+1):
                initial_state = state[1].strip()
                
        condition = f'V{i+1}(state{{{initial_state}}}, {sites})'
        
        # add a comma unless it's the last entry
        if i + 1 < int(mln_data['actors'][1]):
            condition += ','
        
        i_c.append(condition)
    
    i_c.append(')')
        
    i_c = '\n'.join(i_c)
    
    return i_c

In [65]:
print(parse_initial_conditions(mln_data, language))

/* Initial conditions */
%init: 1000 (
V1(state{AI}, l1v2[112], l1v3[113], l2v2[212], l2v3[213]),
V2(state{US}, l1v1[112], l2v1[212], l2v3[223]),
V3(state{US}, l1v1[113], l2v1[213], l2v2[223])
)


In [121]:
def parse_rules(mln_data, language):
    kappa_rules = ['/* Rules */']
    
    rules = [x.split('@') for x in language['rules']]
    rules_organised = []
    for ruleset in rules:
        rules_organised.append({'rule': ruleset[0].strip(), 'rate': ruleset[1].strip()})

    for ruleset in rules_organised:
        if '=' not in ruleset['rule']:
            # parse rules not dependant on layers
            rule_states = [state.strip() for state in ruleset['rule'].split('->')]
            kappa_rules.append(f"'{rule_states[0]} to {rule_states[1]}'")
            for i, actor in enumerate(mln_data['actors'][0]):
                kappa_rule = f"V{i+1}(state{{{rule_states[0]}}}) -> V{i+1}(state{{{rule_states[1]}}}) @ '{ruleset['rate']}'"
                kappa_rules.append(kappa_rule)
        else:
            # parse intra-layer rules
            # requires rules in both directions
            rule_sides = [state.strip() for state in ruleset['rule'].split('->')]
            layer = rule_sides[0][rule_sides[0].index('=')+1].strip()

            states = [[x[0].strip(), x[1].strip()] for x in [rule_side.split(f'={layer}') for rule_side in rule_sides]]
            kappa_rules.append(f"'{layer}: {states[0][0]}-{states[0][1]} to {states[1][0]}-{states[1][1]}'")
            for edge in mln_data['edges'][1][int(layer)-1]:
                v1, v2 = edge[0], edge[1]
                kappa_rules.append(f"// V{v1} - V{v2}")
                site_label = f'{layer}{v1}{v2}'
                kappa_rule = (f'V{v1}(state{{{states[0][0]}}}, l{layer}v{v2}[{site_label}]), V{v2}(state{{{states[0][1]}}}, l{layer}v{v1}[{site_label}]) -> '
                    f'V{v1}(state{{{states[1][0]}}}, l{layer}v{v2}[{site_label}]), V{v2}(state{{{states[1][1]}}}, l{layer}v{v1}[{site_label}]) @ '
                    f"'{ruleset['rate']}'")

                kappa_rules.append(kappa_rule)

        kappa_rules.append('')

    kappa_rules = '\n'.join(kappa_rules)

    return kappa_rules

In [122]:
print(parse_rules(mln_data, language))

/* Rules */
'AS to US'
V1(state{AS}) -> V1(state{US}) @ 'delta'
V2(state{AS}) -> V2(state{US}) @ 'delta'
V3(state{AS}) -> V3(state{US}) @ 'delta'

'AI to AS'
V1(state{AI}) -> V1(state{AS}) @ 'mu'
V2(state{AI}) -> V2(state{AS}) @ 'mu'
V3(state{AI}) -> V3(state{AS}) @ 'mu'

'1: AS-AI to AI-AI'
// V1 - V2
V1(state{AS}, l1v2[112]), V2(state{AI}, l1v1[112]) -> V1(state{AI}, l1v2[112]), V2(state{AI}, l1v1[112]) @ 'betaA'
// V2 - V1
V2(state{AS}, l1v1[121]), V1(state{AI}, l1v2[121]) -> V2(state{AI}, l1v1[121]), V1(state{AI}, l1v2[121]) @ 'betaA'
// V1 - V3
V1(state{AS}, l1v3[113]), V3(state{AI}, l1v1[113]) -> V1(state{AI}, l1v3[113]), V3(state{AI}, l1v1[113]) @ 'betaA'
// V3 - V1
V3(state{AS}, l1v1[131]), V1(state{AI}, l1v3[131]) -> V3(state{AI}, l1v1[131]), V1(state{AI}, l1v3[131]) @ 'betaA'

'1: US-AI to AI-AI'
// V1 - V2
V1(state{US}, l1v2[112]), V2(state{AI}, l1v1[112]) -> V1(state{AI}, l1v2[112]), V2(state{AI}, l1v1[112]) @ 'betaU'
// V2 - V1
V2(state{US}, l1v1[121]), V1(state{AI}, l1v2[

In [136]:
parser.parse_to_kappa('network.txt', 'toy-example.mln', 'out-kappa.txt')

Parsing network configuration file network.txt
Parsing MLN language file toy-example.mln
Successfully exported model into Kappa: out-kappa.txt
